# Rupaul's Drag Race Machine

In [2]:
import pandas as pd
from scipy.stats import rankdata, kendalltau

theData = pd.read_csv("dragrace.csv")
theData= theData.fillna(0)
#theData.iloc[1:10,:]

In [3]:
queens = theData.groupby('Name').max()
queens = queens.reset_index(drop=False)
#queens.iloc[1:10,:]

In [4]:
#queens.loc[queens.Place==1,:]

In [5]:
Xtrain = queens.loc[queens.Season<7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
Xtest = queens.loc[queens.Season==7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
ytrain = queens.loc[queens.Season<7,'Place']
ytest = queens.loc[queens.Season==7,'Place']
season8X = queens.loc[queens.Season==8,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
season8y = queens.loc[queens.Season==8,'Place']
season7 = queens.loc[queens.Season==7,['Name','Place']]
season7.columns = ['Name','Actual']
season7 = season7.reset_index(drop=True)
season8 = queens.loc[queens.Season==8,['Name','Place']]
season8.columns = ['Name','Actual']
season8 = season8.reset_index(drop=True)

# Support Vector Machine Classifier

In [6]:
from sklearn.svm import SVC
model = SVC()
model.fit(Xtrain,ytrain)

yfit = model.predict(Xtest)
yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')
svc7 = pd.merge(season7,yfitpd).sort_values('Actual')
svc7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,5
9,Pearl,3,9
5,Kennedy Davenport,4,5
4,Katya,5,5
12,Trixie Mattel,6,9
7,Miss Fame,7,3
1,Jaidynn Diore Fierce,8,12
6,Max,9,9
3,Kandy Ho,10,4


In [7]:
kendalltau(svc7.Actual,svc7.Predicted)

KendalltauResult(correlation=0.22130457664129397, pvalue=0.27024792506861006)

## Season 8 Predictions

In [8]:
presentFit = model.predict(season8X)
presentFitPD = pd.DataFrame(presentFit)
presentFitPD['Name'] = season8['Name']
presentFitPD.columns = ['Predicted','Name']
presentFitPD['Predicted'] = rankdata(presentFitPD.Predicted,method='min')
svc8 = pd.merge(season8,presentFitPD).sort_values('Predicted')
svc8

,Name,Actual,Predicted
0,Acid Betty,8,1
8,Naomi Smalls,0,2
1,Bob the Drag Queen,0,3
2,Chi Chi DeVayne,0,3
4,Dax ExclamationPoint,11,3
5,Derrick Barry,0,3
9,Naysha Lopez,9,3
10,Robbie Turner,7,3
3,Cynthia Lee Fontaine,10,9
11,Thorgy Thor,0,9


In [10]:
svc8tau = svc8.loc[svc8.Actual!=0,:]
kendalltau(svc8tau.Actual,svc8tau.Predicted)

KendalltauResult(correlation=0.54006172486732174, pvalue=0.12803440781219619)

# Gaussian Naive Bayes

In [11]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(Xtrain,ytrain)
nbTest = model.predict(Xtest)
nbTestDB = pd.DataFrame(nbTest)
nbTestDB['Name'] = season7.Name
nbTestDB.columns = ['Predicted','Name']
nbTestDB = nbTestDB.loc[:,['Name','Predicted']]
nbTestDB['Predicted'] = rankdata(nbTestDB.Predicted,method='min')

nb7 = pd.merge(season7,nbTestDB).sort_values('Actual')
nb7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,8
5,Kennedy Davenport,4,1
4,Katya,5,1
12,Trixie Mattel,6,8
7,Miss Fame,7,4
1,Jaidynn Diore Fierce,8,6
6,Max,9,8
3,Kandy Ho,10,4


In [12]:
kendalltau(nb7.Actual,nb7.Predicted)

KendalltauResult(correlation=0.48108780653777095, pvalue=0.016544547824374332)

## Season 8 Predictions

In [13]:
nb8 = model.predict(season8X)
nb8DB = pd.DataFrame(nb8)
nb8DB['Name'] = season8.Name
nb8DB.columns = ['Predicted','Name']
nb8DB['Predicted'] = rankdata(nb8DB.Predicted,method='min')
nb8 = pd.merge(season8,nb8DB).sort_values('Predicted')
nb8

,Name,Actual,Predicted
0,Acid Betty,8,1
1,Bob the Drag Queen,0,1
6,Kim Chi,0,1
8,Naomi Smalls,0,1
11,Thorgy Thor,0,1
5,Derrick Barry,0,6
3,Cynthia Lee Fontaine,10,7
9,Naysha Lopez,9,7
2,Chi Chi DeVayne,0,9
10,Robbie Turner,7,9


In [14]:
nb8tau = nb8.loc[nb8.Actual!=0,:]
kendalltau(nb8tau.Actual,nb8tau.Predicted)

KendalltauResult(correlation=0.51887452166277093, pvalue=0.14368827622102029)

# Random Forest Classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(Xtrain,ytrain)
yfit = model.predict(Xtest)

yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')

## Season 7 Predictions

In [16]:
rfClass = pd.merge(season7,yfitpd).sort_values('Actual')
rfClass

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,3
5,Kennedy Davenport,4,4
4,Katya,5,4
12,Trixie Mattel,6,6
7,Miss Fame,7,10
1,Jaidynn Diore Fierce,8,6
6,Max,9,1
3,Kandy Ho,10,6


In [17]:
kendalltau(rfClass.Actual,rfClass.Predicted)

KendalltauResult(correlation=0.64061851133006142, pvalue=0.0014157354382150771)

## Season 8 Predictions

In [18]:
rf8Fit = model.predict(season8X)
rf8FitPD = pd.DataFrame(rf8Fit)
rf8FitPD['Name'] = season8.Name
rf8FitPD.columns = ['Predicted','Name']
rf8FitPD = rf8FitPD.loc[:,['Name','Predicted']]
rf8FitPD['Predicted'] = rankdata(rf8FitPD.Predicted,method='min')

In [19]:
rfClass8 = pd.merge(season8,rf8FitPD).sort_values('Predicted')
rfClass8

,Name,Actual,Predicted
0,Acid Betty,8,1
1,Bob the Drag Queen,0,1
10,Robbie Turner,7,3
6,Kim Chi,0,4
2,Chi Chi DeVayne,0,5
5,Derrick Barry,0,6
8,Naomi Smalls,0,6
11,Thorgy Thor,0,6
9,Naysha Lopez,9,9
4,Dax ExclamationPoint,11,10


In [20]:
rfClass8tau = rfClass8.loc[rfClass8.Actual!=0,:]
kendalltau(rfClass8tau.Actual,rfClass8tau.Predicted)

KendalltauResult(correlation=0.57142857142857162, pvalue=0.1073343449848299)

# Random Forest Regressor

In [21]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(200,random_state=24601)
forest.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=24601,
           verbose=0, warm_start=False)

## Season 7 Predictions

In [22]:
forestFit = forest.predict(Xtest)
forestFitDB = pd.DataFrame(forestFit.round())
forestFitDB['Name'] = season7.Name
forestFitDB.columns = ['Predicted','Name']
forestFitDB['Predicted'] = rankdata(forestFitDB.Predicted,method='min')
rfReg7 = pd.merge(season7,forestFitDB).sort_values('Actual')
rfReg7

,Name,Actual,Predicted
13,Voilet Chachki,1,2
0,Ginger Minj,2,1
9,Pearl,3,6
5,Kennedy Davenport,4,2
4,Katya,5,2
12,Trixie Mattel,6,8
7,Miss Fame,7,8
1,Jaidynn Diore Fierce,8,6
6,Max,9,5
3,Kandy Ho,10,8


In [23]:
kendalltau(rfReg7.Actual,rfReg7.Predicted)

KendalltauResult(correlation=0.73379938570534309, pvalue=0.00025655048033250812)

## Season 8 Predictions

In [25]:
forest8 = forest.predict(season8X)
forest8DB = pd.DataFrame(forest8.round())
forest8DB['Name'] = season8.Name
forest8DB.columns = ['Predicted','Name']
forest8DB['Predicted'] = rankdata(forest8DB.Predicted,method='min')
forest8 = pd.merge(season8,forest8DB).sort_values('Predicted')
forest8

,Name,Actual,Predicted
1,Bob the Drag Queen,0,1
6,Kim Chi,0,2
10,Robbie Turner,7,2
2,Chi Chi DeVayne,0,4
5,Derrick Barry,0,4
8,Naomi Smalls,0,4
11,Thorgy Thor,0,4
0,Acid Betty,8,8
7,Laila McQueen,11,9
4,Dax ExclamationPoint,11,10


In [26]:
forest8tau = forest8.loc[forest8.Actual!=0,:]
kendalltau(forest8tau.Actual,forest8tau.Predicted)

KendalltauResult(correlation=0.50000000000000011, pvalue=0.15883380278735759)

# Neural Network

Unfortunately, scikit learn's stable release does not include neural networks (it is being added in the next version) so if I want to include a neural network I have to code one from scratch (credit goes to Joel Grus's Data Science From Scratch book, available from O'Reilly, for much of the code for the neural network)

In [41]:
import numpy as np
import math, random
# create the "step" function
def sigmoid(t): 
    return 1 / (1 + math.exp(-t))

# create the neuron
def neuron_output(weights, inputs):   #This is a simpler representation; weights for input plus one extra (bias)
    return sigmoid(np.dot(weights, inputs))

# define the network
def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""

    outputs = []

    for layer in neural_network: #Remember the neural network is given as a list of "layers" which have neurons in them

        input_with_bias = input_vector + [1]          # add a bias input (this just allos us to use a dot product)
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output

    return outputs

# define the back-propagation that allows the network to learn
def backpropagate(network, input_vector, target):

    hidden_outputs, outputs = feed_forward(network, input_vector)
    
    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]
                     
    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            output_neuron[j] -= output_deltas[i] * hidden_output

    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) * 
                      np.dot(output_deltas, [n[i] for n in network[-1]]) 
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

In [42]:
random.seed(0)   # to get repeatable results
input_size = 8  # each input is a vector of length 8
num_hidden = 5   # we'll have 5 neurons in the hidden layer
output_size = 14 # we need 14 outputs for each input, since there are 14 possible places

In [43]:
# each hidden neuron has one weight per input, plus a bias weight
hidden_layer = [[random.random() for __ in range(input_size + 1)]
                    for __ in range(num_hidden)]

In [44]:
# each output neuron has one weight per hidden neuron, plus a bias weight
output_layer = [[random.random() for __ in range(num_hidden + 1)]
                    for __ in range(output_size)]

In [45]:
# the network starts out with random weights
network = [hidden_layer, output_layer]

Neural networks work best if values are standardized close to 0-1. So for the non-dummy variables, I'll standardize to mean=0, sd=1

In [46]:
# create a function to scale the data for us
def scaleQueens(df):
    """Scale Age, Wins, Highs, Lows, and Lipsyncs in feature data frames"""
    df['Age'] = scale(df['Age'])
    df['Wins'] = scale(df['Wins'])
    df['Highs'] = scale(df['Highs'])
    df['Lows'] = scale(df['Lows'])
    df['Lipsyncs'] = scale(df['Lipsyncs'])
    return df

In [47]:
from sklearn.preprocessing import scale
queensNN = queens
#scale season for entire data set (I'll scale each subset of data individually for everything else)
queensNN['Season'] = scale(queensNN['Season'])

#dataset = zip(inputs_dig, targets_dig)

In [48]:
XtrainNN = queensNN.loc[queens.Season<7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
XtrainNN = scaleQueens(XtrainNN)
XtestNN = queensNN.loc[(queensNN.Season>1) & (queensNN.Season<1.5),['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
XtestNN = scaleQueens(XtestNN)
ytrainNN = queensNN.loc[queens.Season<7,'Place']
ytestNN = queensNN.loc[queens.Season==7,'Place']
season8XNN = queensNN.loc[queensNN.Season>1.5,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
season8XNN = scaleQueens(season8XNN)
season8yNN = queensNN.loc[queens.Season==8,'Place']

In [49]:
yNN = [[1 if i == j else 0 for i in range(14)]
               for j in ytrainNN ]
XNN = XtrainNN.values.tolist()

In [50]:

for i in range(10000):
    for input_vector, target_vector in zip(XNN, yNN):
        backpropagate(network, input_vector, target_vector)

In [51]:
def predict(input):
        return feed_forward(network, input)[-1]

## Season 7 Predictions

In [52]:
X7NN = XtestNN.values.tolist()

In [53]:
predictedY = []
for i, input in enumerate(X7NN):
    outputs = predict(input)
    predictedY.append(outputs)
#for q in predictedY:
#    print([round(a,2) for a in q])

In [54]:
predictedPlace = [a.index(max(a)) for a in predictedY]
predictedPlace = rankdata(predictedPlace,method='min')
season7NN = season7
season7NN['Predicted'] = predictedPlace
season7NN.sort_values('Actual')

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,5
9,Pearl,3,2
5,Kennedy Davenport,4,3
4,Katya,5,3
12,Trixie Mattel,6,8
7,Miss Fame,7,6
1,Jaidynn Diore Fierce,8,7
6,Max,9,14
3,Kandy Ho,10,8


In [55]:
kendalltau(season7NN.Actual,season7NN.Predicted)

KendalltauResult(correlation=0.66575028593568286, pvalue=0.00091111916489375333)

## Season 8 Predictions

In [56]:
season8XNN['Wins'] = scale(season8XNN['Wins'])
season8XNN['Highs'] = scale(season8XNN['Highs'])
season8XNN['Lows'] = scale(season8XNN['Lows'])
season8XNN['Lipsyncs'] = scale(season8XNN['Lipsyncs'])
X8NN = season8XNN.values.tolist()
predictedY = []
for i, input in enumerate(X8NN):
    outputs = predict(input)
    predictedY.append(outputs)
for q in predictedY:
    print([round(a,2) for a in q])

[0.91, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.26, 0.0, 0.0, 0.0, 0.0, 0.03]
[0.03, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.96, 0.0, 0.0, 0.01, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.04, 0.13, 0.13, 0.36, 0.0, 0.06]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.04, 0.13, 0.13, 0.36, 0.0, 0.06]
[0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.2, 0.82, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0]
[0.43, 0.46, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.04, 0.13, 0.13, 0.36, 0.0, 0.06]
[0.99, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.04, 0.13, 0.13, 0.36, 0.0, 0.06]
[0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.2, 0.82, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0]
[0.97, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.28, 0.0, 0.0, 0.0, 0.0, 0.03]


In [57]:
[a.index(max(a)) for a in predictedY]

[0, 1, 7, 11, 11, 7, 1, 11, 0, 11, 7, 0]

In [59]:
predictedPlace = [a.index(max(a)) for a in predictedY]
predictedPlace = rankdata(predictedPlace,method='min')
season8NN = season8
season8NN['Predicted'] = predictedPlace
season8NN = season8NN.sort_values('Predicted')
season8NN

,Name,Actual,Predicted
0,Acid Betty,8,1
8,Naomi Smalls,0,1
11,Thorgy Thor,0,1
1,Bob the Drag Queen,0,4
6,Kim Chi,0,4
2,Chi Chi DeVayne,0,6
5,Derrick Barry,0,6
10,Robbie Turner,7,6
3,Cynthia Lee Fontaine,10,9
4,Dax ExclamationPoint,11,9


In [60]:
season8tau = season8NN.loc[season8NN.Actual!=0,:]
kendalltau(season8tau.Actual,season8tau.Predicted)

KendalltauResult(correlation=0.62360956446232352, pvalue=0.07886029754662148)

# Evolutionary Neural Network

Evolutionary neural networks allow neural networks to evolve over time, gaining complexity and (hopefully) precision through a combination of random mutations and reproduction of "successful" genomes. Here I use neat-python to generate neural networks through a process called NeuroEvolution of Augmenting Topologies. 

In [65]:
from neat import  nn, population, statistics, visualize

/Users/thomas/anaconda/lib/python3.5/site-packages/neat/visualize.py:12: UserWarning: Could not import optional dependency graphviz.
  warnings.warn('Could not import optional dependency graphviz.')


In [66]:
import math
def eval_fitness(genomes):
    for g in genomes:
        net = nn.create_feed_forward_phenotype(g)

        sum_square_error = 0.0
        predictions = []
        for inputs, expected in zip(XNN, yNN):
            # Serial activation propagates the inputs through the entire network.
            output = net.serial_activate(inputs)
            #sum_square_error += (output[0] - expected) ** 2
            predictions.append(output)
        predictedPlace = [a.index(max(a)) for a in predictions]
        actualPlace = [a.index(max(a)) for a in yNN]
        t = kendalltau(predictedPlace,actualPlace)[0]
        if math.isnan(t):
            t = 0
        g.fitness = t

In [67]:
a = [1,2,3,4,5]
b = [1,1,1,1,1]
kendalltau(a,b)[1]

nan

In [68]:
pop = population.Population('dragconfig')

In [69]:
pop.run(eval_fitness,300)


 ****** Running generation 0 ****** 

Population's average fitness: 0.02687 stdev: 0.23167
Best fitness: 0.46065 - size: (5, 19) - species 1 - id 3
Species length: 1 totaling 150 individuals
Species ID       : [1]
Species size     : [150]
Species age      : [0]
Species no improv: {1: 0}
Average adjusted fitness: 0.000
Spawn amounts: [150]
Species fitness  : [0.00017916470821847079]
Generation time: 0.994 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.17455 stdev: 0.36261
Best fitness: 0.48709 - size: (5, 19) - species 1 - id 160
Species length: 1 totaling 150 individuals
Species ID       : [1]
Species size     : [150]
Species age      : [1]
Species no improv: {1: 0}
Average adjusted fitness: 0.001
Spawn amounts: [150]
Species fitness  : [0.0011636976799741445]
Generation time: 0.909 sec

 ****** Running generation 2 ****** 

Population's average fitness: 0.22362 stdev: 0.36332
Best fitness: 0.48935 - size: (5, 19) - species 2 - id 420
Species length: 2 tota

In [70]:
winner = pop.statistics.best_genome()
print('\nBest genome:\n{!s}'.format(winner))


Best genome:
Nodes:
	NodeGene(id=0, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=1, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=2, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=3, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=4, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=5, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=6, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=7, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=8, type=OUTPUT, bias=-3.2914017756698883, response=5.511966833677718, activation=sigmoid)
	NodeGene(id=9, type=OUTPUT, bias=-2.273702209954699, response=5.111714868783216, activation=sigmoid)
	NodeGene(id=10, type=OUTPUT, bias=-0.2889235398649022, response=5.335463261278775, activation=sigmoid)
	NodeGene(id=11, type=OUTPUT, bias=0.30290367375670924

In [71]:
winner_net = nn.create_feed_forward_phenotype(winner)

predicted = []
for inputs in XNN:
    output = winner_net.serial_activate(inputs)
    predicted.append(output)
predictedPlace = [a.index(max(a)) for a in predicted]
actualPlace = [a.index(max(a)) for a in yNN]
kendalltau(predictedPlace,actualPlace)[0]

0.74827585485676562

## Season 7 Predictions

In [72]:
predicted = []
for inputs in X7NN:
    output = winner_net.serial_activate(inputs)
    predicted.append(output)
predictedPlace = [a.index(max(a)) for a in predicted]
predictedPlace = rankdata(predictedPlace,method='min')
season7NN = season7
season7NN['Predicted'] = predictedPlace
season7NN.sort_values('Actual')

,Name,Actual,Predicted
13,Voilet Chachki,1,2
0,Ginger Minj,2,2
9,Pearl,3,5
5,Kennedy Davenport,4,5
4,Katya,5,5
12,Trixie Mattel,6,5
7,Miss Fame,7,5
1,Jaidynn Diore Fierce,8,10
6,Max,9,2
3,Kandy Ho,10,11


In [73]:
kendalltau(season7NN.Actual,season7NN.Predicted)

KendalltauResult(correlation=0.42004127944129094, pvalue=0.036388871175315939)